In [1]:
import pandas as pd

In [2]:
last_week = pd.read_excel(r"C:\Users\pa173\Documents\SQL Excel Extracts\Weekly Producer\January\Jan 15 - Jan 21 2023.xlsx", sheet='Export Worksheet', index=False)

In [3]:
all_parent = pd.read_excel(r"C:\Users\pa173\Documents\Communications\All Producer with Parent Rollup.xlsx", index=False)

In [4]:
rank_list = pd.read_excel(r"C:\Users\pa173\Documents\Communications\Wkly Working - Rank and List.xlsx", index=False)

In [5]:
last_report = pd.read_excel(r"C:\Users\pa173\Documents\Communications\Weekly NB and RW Apps and FQ.xlsx", sheet_name='By Producer Weekly Counts', index=False, skiprows= 9)
last_report = last_report.drop(index=0)

In [6]:
nb_rw_pivot = pd.read_excel(r"C:\Users\pa173\Documents\Communications\Weekly NB and RW Apps and FQ.xlsx", sheet_name='Pivot', index=False, skiprows= 6)

In [7]:
def modify_cols(df):
    df.columns=df.columns.str.lower()
    df.columns=df.columns.str.strip()
    df.columns=df.columns.str.replace(" ", "_")
    return df


In [8]:
last_week=modify_cols(last_week)
all_parent=modify_cols(all_parent)
rank_list=modify_cols(rank_list)
last_report=modify_cols(last_report)
nb_rw_pivot=modify_cols(nb_rw_pivot)

merging last weeks data with all producers

In [9]:
parent_week=pd.merge(left=last_week, right=all_parent, left_on='sub_business_unit_number', right_on='business_unit_number', how='left')

In [10]:
parent_week.shape

(471, 15)

In [11]:
producer_pivot=parent_week.pivot_table(index=['parent_name'], values=['count(*)'], aggfunc='sum')
#producer_pivot = parent_week.pivot_table(index='parent_name', values='count(*)', columns=['parent_name','count(*)'], aggfunc='sum').sort_values(['count(*)'], ascending=False)

In [13]:
producer_pivot.shape

(457, 1)

In [14]:
rank_list.shape

(1012, 3)

In [16]:
producer_rank=pd.merge(left=producer_pivot, right=rank_list, left_on='parent_name', right_on='producer', how='left') # NOT SURE WHY BUT IT HAS TO BE MERGED THIS WAY TO WORK

In [17]:
producer_rank.shape

(457, 4)

cleaning nb and rw pivots

In [18]:
nb_pivot = nb_rw_pivot[['row_labels', 'unnamed:_8']]
nb_pivot=nb_pivot.rename(columns={'row_labels':'nb_labels','unnamed:_8':'nb_total'})

In [19]:
rw_pivot = nb_rw_pivot[['row_labels.1', 'unnamed:_19']]
rw_pivot=rw_pivot.rename(columns={'row_labels.1':'rw_labels','unnamed:_19':'rw_total'})

In [20]:
rw_pivot=rw_pivot.dropna()

In [21]:
nb_pivot.shape, rw_pivot.shape

((205, 2), (104, 2))

clean last weeks report

In [22]:
last_report = last_report.iloc[:, 1:17]
last_report.iloc[:,4:16] = last_report.iloc[:, 4:16].apply(pd.to_numeric)
last_report= last_report.rename(columns = {'weekly_counts_by_week': 'producer'})

merge parent pivot with last weeks report

In [23]:
new_report=pd.merge(last_report, producer_rank, left_on='producer', right_on='producer', how='left')

In [24]:
new_report.shape

(1015, 19)

merge new report with nb and rw tables

In [25]:
new_report=pd.merge(new_report, nb_pivot, left_on='producer', right_on='nb_labels', how='left')

In [26]:
new_report.shape

(1015, 21)

In [27]:
new_report=pd.merge(new_report, rw_pivot, left_on='producer', right_on='rw_labels', how='left')

In [28]:
new_report.shape

(1015, 23)

In [29]:
new_report=new_report.fillna(0)

In [30]:
new_report['incl']=new_report['incl'].replace(0,'N')

updating our changes

In [31]:
new_report['change_quotes']=new_report['count(*)'] - new_report['quotes.1']

In [32]:
new_report['change_nb_apps']=new_report['nb_total'] - new_report['nb_apps.1']

In [33]:
new_report['change_rw_apps']=new_report['rw_total'] - new_report['rw_apps.1']

In [36]:
new_report.to_excel(r"C:\Users\pa173\Documents\Python\Weekly Report\new_report.xlsx", index=False)